In [2]:
import json
import pandas as pd

In [119]:
partidos_votos = json.load(open("partidos_votos_total.json"))

In [120]:
def json_to_df(votos, column_name):
    all_props = []
    for row in votos:
        party_name = row["nome"]
        propositions = row['temas']
        for prop in propositions:
            dict_prop = {column_name:party_name}
            dict_prop.update(prop)
            all_props.append(dict_prop)
    all_props = pd.DataFrame(all_props)
    return all_props

In [121]:
all_props = json_to_df(partidos_votos, "party_name")
props_by_party = all_props[["party_name", "tema", "value"]].pivot(index="party_name", columns='tema', values='value').to_dict('index')

In [122]:
no_vote = "-10"
def calc_sim(propsA, propsB):
    equals = 0
    to_sum = 0
    for prop, vA in propsA.iteritems():
        if vA == no_vote:
            continue
        vB = propsB[prop]
        if vB == no_vote:
            continue
        if vA == vB:
            equals +=1
        to_sum+=1
    if to_sum == 0:
        return None, to_sum
    else:
        return equals / float(to_sum), to_sum
assert calc_sim({"a": 1, "b": 0}, {"a": 1, "b": 0}) == (1, 2)
assert calc_sim({"a": 1, "b": 0}, {"a": 1, "b": 1}) == (.5, 2)
assert calc_sim({"a": 1, "b": 0}, {"a": 0, "b": 1}) == (0.0, 2)
assert calc_sim({"a": 1, "b": 0}, {"a": 1, "b": no_vote}) == (1.0, 1)
assert calc_sim({"a": 1, "b": 0}, {"a": no_vote, "b": no_vote}) == (None, 0)

In [123]:
all_sim = []
parties_already_calculates = set()

def get_key(partyA, partyB):
    return "{}_{}".format(partyA, partyB)

for partyA, propsA in props_by_party.iteritems():
    for partyB, propsB in props_by_party.iteritems():
        if get_key(partyA, partyB) in parties_already_calculates or \
            get_key(partyB, partyA) in parties_already_calculates:
            continue

        if partyA == partyB:
            sim, used = 1,1
        else:
            sim, used = calc_sim(propsA, propsB)
        
        all_sim.append({"source": partyA, "target": partyB, "value": sim, "used": used})
        if partyA != partyB:
            all_sim.append({"target": partyA, "source": partyB, "value": sim, "used": used})
        parties_already_calculates.add(get_key(partyA, partyB))
        parties_already_calculates.add(get_key(partyB, partyA))

In [130]:
all_sim_df[all_sim_df.source == "pcb"]

,source,target,used,value


In [127]:
all_sim_df = pd.DataFrame(all_sim)
all_sim_df[all_sim_df.source == "pros"].sort_values(by="value", ascending=False)

,source,target,used,value
700,pros,pros,1,1.000000
725,pros,avante,1,1.000000
148,pros,prb,24,0.666667
713,pros,psl,24,0.666667
711,pros,ptdob,23,0.652174
364,pros,prp,20,0.650000
28,pros,ptn,22,0.636364
205,pros,ptc,19,0.631579
460,pros,pmn,19,0.631579
707,pros,psdc,19,0.631579


In [100]:
 set(all_sim_df[(all_sim_df.used >= 5)].source.drop_duplicates()) - set(all_sim_df[(all_sim_df.used >= 5) & (all_sim_df.value >= 0.75)].source.drop_duplicates())

{u'pps', u'pros', u'psb', u'pv'}

In [101]:
set(all_sim_df.source.drop_duplicates()) - set(all_sim_df[(all_sim_df.used >= 5)].source.drop_duplicates())

{u'avante', u'pmb', u'podemos'}

In [108]:
temp = all_sim_df[(all_sim_df.used >= 5) & (all_sim_df.value >= 0.75)]
all_sim = temp.to_dict('records')
parties = temp.source.drop_duplicates()
parties_without_similarity =  set(all_sim_df[(all_sim_df.used >= 5)].source.drop_duplicates()) - \
set(all_sim_df[(all_sim_df.used >= 5) & (all_sim_df.value >= 0.75)].source.drop_duplicates())
parties = list(parties) + list(parties_without_similarity)

In [109]:
len(all_sim)

158

In [110]:
for i, s in enumerate(all_sim):
    all_sim[i]['source'] = nodes_name_to_index[all_sim[i]['source']]
    all_sim[i]['target'] = nodes_name_to_index[all_sim[i]['target']]

In [114]:
def get_group(party):
    if party in ["psol", "pt", "pdt", "pcdob", "rede"]:
        return "red"
    elif party in ["dem", "solidaried", "psdb"]:
        return "blue"
    elif party in ["ptn", "prp", "pmn", "prtb", "ptdob", "ptc", "psdc"]:
        return "green"
    elif party in ["prb", "psl", "pp", "ptb", "pmdb", "psd", "pen", "pr", "phs", "psc"]:
        return "orange"
    else:
        return "black"
nodes = [{"id": n, "group": get_group(n)} for n in parties]
nodes_name_to_index = {n['id']: i for i,n in enumerate(nodes)}

    
links = all_sim

data = {"nodes": nodes,
        "links": links}

json.dump(data, open("analyses/data.json", "w"))

# Compare parties and deputies

In [4]:
partidos_votos = json.load(open("data/partidos_votos_total.json"))
deputados_votos = json.load(open("data/deputados_votos_total.json"))

In [5]:
deputados_votos

[{u'id_dep': u'178957',
  u'nome': u'Abel Mesquita Jr.',
  u'partido': u'dem',
  u'temas': [{u'tema': u'Cobran\xe7a de cursos em universidades p\xfablicas',
    u'value': u'0',
    u'value_name': u'n\xe3o'},
   {u'tema': u'Coincid\xeancia reelei\xe7\xe3o',
    u'value': u'1',
    u'value_name': u'sim'},
   {u'tema': u'Cota para mulheres legislativo',
    u'value': u'1',
    u'value_name': u'sim'},
   {u'tema': u'Distritao', u'value': u'0', u'value_name': u'n\xe3o'},
   {u'tema': u'Financiamento privado para partidos',
    u'value': u'1',
    u'value_name': u'sim'},
   {u'tema': u'Financiamento privado para partidos e candidatos',
    u'value': u'1',
    u'value_name': u'sim'},
   {u'tema': u'Impeachment', u'value': u'1', u'value_name': u'sim'},
   {u'tema': u'Infatic\xeddio ind\xedgena',
    u'value': u'1',
    u'value_name': u'sim'},
   {u'tema': u'Lei trabalhista', u'value': u'1', u'value_name': u'sim'},
   {u'tema': u'Maioridade 1', u'value': u'0', u'value_name': u'n\xe3o'},
   {u't

In [43]:
party_props = json_to_df(partidos_votos, "party_name")
props_by_party = party_props[["party_name", "tema", "value"]].pivot(index="party_name", columns='tema', values='value').to_dict('index')

In [44]:
deputies_props = json_to_df(deputados_votos, "deputy_name")
props_by_deputy = deputies_props[["deputy_name", "tema", "value"]].pivot(index="deputy_name", columns='tema', values='value').to_dict('index')

In [53]:
part_by_deputy = {d['nome']: d['partido'] for d in deputados_votos}

In [82]:
no_vote_A = "-1"
no_vote_B = "-10"
ABSTENCAO = "-2"
OBSTRUCAO = "-3"
ARTIGO_17 = "-4"
PARTY_LIBERADO = "-1"
def calc_sim(propsA, propsB):
    equals = 0
    to_sum = 0
    for prop, vA in propsA.iteritems():
        vA = str(vA)
        if vA in [no_vote_A, ABSTENCAO, ARTIGO_17]: #SÓ PASSA 0, 1, -3
            continue
        if vA == OBSTRUCAO: #SÓ PASSA 0, 1
            vA = "0"
        vB = str(propsB[prop])
        if vB == no_vote_B: #SÓ PASSA 0, 1, -1
            continue
        if vB == PARTY_LIBERADO:
            equals += 1
            to_sum += 1
        elif vA == vB and vA in ["1", "0"]:
            equals +=1
            to_sum+=1
        else:
            to_sum+=1
            if vA not in ["1", "0"] and vB not in ["1", "0"]:
                print(vA, vB)
    if to_sum == 0:
        return None, to_sum
    else:
        return equals / float(to_sum), to_sum
assert calc_sim({"a": 1, "b": 0}, {"a": 1, "b": 0}) == (1, 2)
assert calc_sim({"a": 1, "b": 0}, {"a": 1, "b": 1}) == (.5, 2)
assert calc_sim({"a": 1, "b": 0}, {"a": 0, "b": 1}) == (0.0, 2)
assert calc_sim({"a": 1, "b": 0}, {"a": 1, "b": no_vote}) == (1.0, 1)
assert calc_sim({"a": 1, "b": 0}, {"a": no_vote, "b": no_vote}) == (None, 0)

In [83]:
all_sim = []
def get_key(partyA, partyB):
    return "{}_{}".format(partyA, partyB)

for deputy, props_deputy in props_by_deputy.iteritems():
    party = part_by_deputy[deputy]
    props_party = props_by_party[party]
    sim, used = calc_sim(props_deputy, props_party)
        
    all_sim.append({"source": deputy, "target": party, "value": sim, "used": used})

In [85]:
df = pd.DataFrame(all_sim)

In [91]:
df[df.used >= 5].value.describe()

count    523.000000
mean       0.867719
std        0.125266
min        0.235294
25%        0.800000
50%        0.904762
75%        0.957428
max        1.000000
Name: value, dtype: float64